In [ ]:
# Load library imports
import sys
import torch
import random
import logging
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import box
from shapely.geometry import Point

# Load project Imports
from src.utils.config_loader import load_project_config
from src.visualisation.mapped_visualisations import plot_interactive_mesh
from src.data_ingestion.landcover_data_ingestion import load_land_cover_data

In [ ]:
# Set up logger config
logging.basicConfig(
    level=logging.INFO,
   format='%(levelname)s - %(message)s',
#    format='%(asctime)s - %(levelname)s - %(name)s - %(message)s',
    handlers=[logging.StreamHandler(sys.stdout)]
)

# Set up logger for file and load config file for paths and params
logger = logging.getLogger(__name__)
config = load_project_config(config_path="config/project_config.yaml")
notebook = True

# Set up seeding to define global states
random_seed = config["global"]["pipeline_settings"]["random_seed"]
random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Define notebook demo catchment
catchments_to_process = config["global"]["pipeline_settings"]["catchments_to_process"]
catchment = catchments_to_process[0]
run_defra_API_calls = config["global"]["pipeline_settings"]["run_defra_api"]  # True to run API calls

logger.info(f"Show Notebook Outputs: {notebook}")
logger.info(f"Notebook Demo Catchment: {catchment.capitalize()}")

Note: CAMELS-GB is a rich data resource at the correct resolution but the time series data ends in 2015 so is insufficient for this model. Some of the static features in the dataset are still applicable:

- Land cover (e.g., % Urban, % Arable, etc.)
- Elevation and slope (from DEMs in the CAMELS dataset)
- Soil type / BFIHOST
- Hydrogeological classifications (e.g., aquifer classes)
- Geographic coordinates (for snapping)

### Land Cover Data Ingestion ###

In [ ]:
# UKCEH LCM2023
land_cover_df, catchment_gdf = load_land_cover_data(
    tif_path='data/01_raw/eden/static/land_cover/gb2023lcm1km_dominant_aggregate.tif',
    csv_path='data/02_processed/eden/static/land_cover/land_cover.csv',
    catchment=catchment,
    shape_filepath=config[catchment]['paths']['gis_catchment_boundary']
)

In [ ]:
display(land_cover_df)

In [ ]:
categories = land_cover_df['land_cover_code'].value_counts()
print(categories)

In [ ]:
logger.info(f"Preparing land cover data for visual check...")

# Convert land_cover_df to a GeoDataFrame and create a geometry column using the 'lon' and 'lat' coordinates
geometry_lc_for_plot = [Point(xy) for xy in zip(land_cover_df['lon'], land_cover_df['lat'])]
land_cover_gdf_for_plot = gpd.GeoDataFrame(land_cover_df.copy(), geometry=geometry_lc_for_plot, crs="EPSG:4326")

# Plot bounding box of data to check spatial geometry (notebook only)
land_cover_map = plot_interactive_mesh(
    mesh_nodes_gdf=land_cover_gdf_for_plot, # This will plot your land cover points
    catchment_polygon=catchment_gdf, # This will plot your catchment boundary
    map_blue=config['global']['visualisations']['maps']['map_blue'],
    esri=config['global']['visualisations']['maps']['esri'],
    esri_attr=config['global']['visualisations']['maps']['esri_attr'],
    static_output_path=f"{config[catchment]['visualisations']['maps']['static_mesh_map_output']}_land_cover",
    interactive_output_path=f"{config[catchment]['visualisations']['maps']['interactive_mesh_map_output']}_land_cover",
    grid_resolution=config[catchment]['preprocessing']['graph_construction']['grid_resolution'],
    interactive=config['global']['visualisations']['maps']['display_interactive_map']
)

# Display map in notebook
land_cover_map

### Elevation Data Ingestion ###

In [ ]:
# DIGIMAPS (via OS Terrain 5 / Terrain 50)

### Slope Data Ingestion ###

In [ ]:
# DEMS / CAMELS-GB

### Soil Type Data Ingestion ###

In [ ]:
# CEH's Grid-to-Grid soil maps / HOST soil classes

### Aquifer Type Data Ingestion ###

In [ ]:
# DEFRA / British Geological Survey (BGS)

### Aquifer Transmissivity Data Ingestion ###

In [ ]:
# DEFRA / British Geological Survey (BGS)

### Geological Maps Data Ingestion ###

In [ ]:
# DIGIMAPS (BGS data via Geology Digimap)

### Distance from River (Derived) ###

In [ ]:
# Derived from DEFRA / DIGIMAP